# PPO（Proximal Policy Optimization，临近策略优化）


## 前置概念 

### Off Policy

在 On Policy 中，采集数据使用的模型（策略）和训练的模型是同一个，每次训练完后都需要重新采集数据，然后重新训练。训练速度很慢。

如果数据采集模型和训练模型不是同一个，并且采集的数据可以被多次用来训练，便可以提升采集速度，称为 Off Policy。

![PPO-1](./images/PPO-1.png)

例如：

老师针对小明的表现表扬或者批评小明，如果老师表扬小明，那么小明加强受表扬的行为，反之亦然。这个过程可以视为一个 On Policy。

其他同学根据老师表扬或批评小明所做的行为，而调整自己的行为，这就是 Off Policy。

### 重要性采样

在 x 服从分布 p 的情况下，求 f(x) 的期望，公式为： 

$ E(f(x))_{x \sim p(x)} = \sum_{x} f(x) * p(x) \approx \frac{1}{N} \sum_{n=1}^{N}f(x)_{x \sim p(x)} $

接下来，乘以并除以 x 在分布 q 下的概率 q(x):

$ = \sum_{x} f(x) * p(x) \frac {q(x)} {q(x)} $

$ = \sum_{x} f(x) \frac {p(x)} {q(x)} * q(x)  $

最后的这个式子，可以看成在 x 服从分布 q 的情况下，求$ f(x) \frac {p(x)} {q(x)}$的期望：

$ = E(f(x) \frac {p(x)} {q(x)})_{x \sim q(x)} $ 

这样，在 q 分布下采样，计算$ f(x) \frac {p(x)} {q(x)}$ 的期望就等于在 p 分布下采样，计算 $f(x)$ 的期望。

再进一步，该式还约等于：

$ \approx \frac{1}{N} \sum_{n=1}^{N}f(x) \frac{p(x)}{q(x)}_{x \sim q(x) }  =   \frac{1}{N} \sum_{n=1}^{N}f(x)_{x \sim p(x)} $

表示为在 x 服从 q 分布下采样 N 条记录后，求均值。

使用重要性采样更新目标函数的梯度公式：

$\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta }^{GAE}(s_{n}^{t},a_{n}^{t}) \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t}) $

$ = \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t}) \frac{P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})}  \nabla \log P_{\theta }(a_{n}^{t}|s_{n}^{t})  $

使用参考策略 $\theta’$ 进行采样，计算其优势函数 $A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t})$，用他来更新训练策略 $\theta$。

这就将 On Policy 训练 转换为 Off Policy 训练。

例如，$\theta’$ 是小明的策略，$\theta$ 是其他学生的策略。$\theta’$ 的优势函数 $A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t})$ 就是老师对小明的批评或者表扬。

你不能直接使用老师对小明的批评或者表扬来更新自己的行为。

假如小明玩手机概率为0.2，你玩手机概率为0.4，在小明受到老师批评后，你不能只下降和小明一样的强度，而应该是小明的两倍才对。

继续推导公式，对数函数的求导公式为：$ \nabla \log f(x) = \frac{\nabla f(x)}{f(x)}  $

带入表达式得：

$ = \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t}) \frac{P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})}  \frac{\nabla P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta }(a_{n}^{t}|s_{n}^{t})} $

$ = \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t}) \frac{\nabla P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})} $

将该梯度公式去掉求梯度符号，再加上负号，就得到了 PPO 的损失函数：
$$ 
Loss= - \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t}) \frac{ P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})}
$$

从公式可以看出，PPO 使用参考的策略 $\theta'$ 来进行数据采样，计算优势函数 $A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t})$。

用训练策略 $\theta$ 做某个动作的概率 除以 参考策略 $\theta'$ 做某个动作的概率来调整优势函数的度，得到运来训练 $\theta$ 网络的损失函数。

用参考策略进行数据采样，采样的数据多次训练 $\theta$ 网络，这就解决了 On Policy 训练效率太低的问题。


需要注意的是，参考策略 $\theta'$ 不应该和训练策略 $\theta$ 在同一情况下，给出各种动作的概率的分布过大。

例如，如果老师对和你差不多的学生进行的表扬或批评，对你来说是由借鉴意义的。如果老师表扬或批评的对象和你差距过大，那就不具备参考意义了。


为此，我们引入 KL 散度（Kullback-Leibler divergence）来约束 训练策略 $\theta$ 与参考策略 $\theta'$ 的动作的概率分布的相似度。

两个策略的动作的概率分布完全一样，KL 散度为 0。分布越不一致，KL 散度就越大。通过 $\beta$ 来控制 KL 散度的大小。

公式为：

$$
Loss_{ppo}= - \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t}) \frac{P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})} + \beta KL(P_{\theta, \theta'})
$$

还有一种方法是通过截断函数来替代 KL 散度，防止训练的策略和参考策略偏差过大。

公式为：
$$
Loss_{ppo2} = - \frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_n} min( A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t}) \frac{ P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})}, clip(\frac{ P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})}), 1-\varepsilon, 1+\varepsilon )A_{\theta' }^{GAE}(s_{n}^{t},a_{n}^{t})
$$

min（取较小值）中，左侧为原始公式，右侧为一个截断函数。

截断函数有三个部分，如果$ \frac{ P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})} $在 $[1-\varepsilon, 1+\varepsilon]$ 之间，就返回其本身的值，如果小于 $1-\varepsilon$，就返回 $1-\varepsilon$，如果大于 $1+\varepsilon$，就返回 $1+\varepsilon$。

这就限制了 $ \frac{ P_{\theta }(a_{n}^{t}|s_{n}^{t})}{P_{\theta '}(a_{n}^{t}|s_{n}^{t})} $ 中二者的不能相差太大。

例如，假设 $ \varepsilon = 0.2 $，如果奖励函数为正值，那么我们需要调整 $ P_{\theta} $，使其尽可能的大，但他和 $ P_{\theta'} $ 的比值不能超过 1.2。

如果奖励函数为负值，那么我们需要调整 $ P_{\theta} $，使其尽可能的小，但他和 $ P_{\theta'} $ 的比值不能小于 0.8。

# 参考
- https://www.bilibili.com/video/BV1iz421h7gb/?spm_id_from=333.337.search-card.all.click&vd_source=8fb10652f8c3316e5308e66bcf6011f0
